In [ ]:
import sqlite3

# Crear una conexión a la base de datos
conn = sqlite3.connect('gloria1.db')

# Crear el cursor
c = conn.cursor()

# Crear la tabla para almacenar las zonas marcadas y las plantillas
c.execute('''CREATE TABLE IF NOT EXISTS ZonasMarcadas (
                id INTEGER PRIMARY KEY,
                plantilla_id INTEGER,
                nombre_zona TEXT,
                coordenadas TEXT
            )''')

# Guardar los cambios
conn.commit()

# Cerrar la conexión
conn.close()


In [4]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox, ttk
from PIL import Image, ImageTk, ImageOps
import fitz
from pytesseract import *

pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


class TextExtractorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Extractor")
        self.root.geometry("1200x500")

        self.frame = tk.Frame(self.root)
        self.frame.pack(padx=20, pady=20, fill=tk.BOTH, expand=True)  

        # Frame para el canvas y scrollbar
        self.canvas_scroll_frame = tk.Frame(self.frame)
        self.canvas_scroll_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)  

        # Canvas para mostrar la imagen
        self.canvas = tk.Canvas(self.canvas_scroll_frame, bg='white', width=800) # Modificar el ancho aquí
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)  

        # Scrollbar para el canvas
        self.scrollbar = ttk.Scrollbar(self.canvas_scroll_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.LEFT, fill=tk.Y)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        # Frame para los botones
        self.buttons_frame = tk.Frame(self.frame)
        self.buttons_frame.pack(side=tk.LEFT, padx=10)  

        self.templates = {}  

        self.btn_select_template = tk.Button(self.buttons_frame, text="Seleccionar Plantilla", command=self.select_template)
        self.btn_select_template.grid(row=0, column=0, pady=10)

        self.btn_mark_zones = tk.Button(self.buttons_frame, text="Marcar Zonas", command=self.mark_zones, state=tk.DISABLED)
        self.btn_mark_zones.grid(row=1, column=0, pady=10)

        self.btn_save_zones = tk.Button(self.buttons_frame, text="Guardar Zonas", command=self.save_zones, state=tk.DISABLED)
        self.btn_save_zones.grid(row=2, column=0, pady=10)

        self.btn_save_template = tk.Button(self.buttons_frame, text="Guardar Plantilla", command=self.save_template, state=tk.DISABLED)
        self.btn_save_template.grid(row=3, column=0, pady=10)

        self.btn_load_file = tk.Button(self.buttons_frame, text="Cargar Archivo", command=self.load_file, state=tk.DISABLED)
        self.btn_load_file.grid(row=4, column=0, pady=10)

        self.template_var = tk.StringVar()
        self.template_var.set("")  

        self.template_menu = tk.OptionMenu(self.buttons_frame, self.template_var, "")
        self.template_menu.grid(row=5, column=0, pady=10)

        self.template_image = None
        self.template_zones = {}
        self.selected_file = None
        self.textboxes = {}
        self.template_ids = {}  # Diccionario para almacenar los plantilla_id

        # Crear la base de datos y la tabla
        conn = sqlite3.connect('gloria1.db')
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS ZonasMarcadas (
                        id INTEGER PRIMARY KEY,
                        plantilla_id INTEGER,
                        nombre_zona TEXT,
                        coordenadas TEXT
                    )''')
        conn.commit()
        conn.close()

    def select_template(self):
        template_file = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if template_file:
            self.load_template(template_file)

    def load_template(self, template_file):
        try:
            doc = fitz.open(template_file)
            page = doc.load_page(0)
            pix = page.get_pixmap()
            self.template_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            self.show_template_image()
            self.btn_mark_zones.config(state=tk.NORMAL)
            self.btn_save_template.config(state=tk.NORMAL)
            self.btn_load_file.config(state=tk.NORMAL)  
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo cargar la plantilla: {e}")

    def show_template_image(self):
        if self.template_image:
            self.canvas.delete("all")
            self.image_tk = ImageTk.PhotoImage(self.template_image)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_tk)
            self.canvas.config(scrollregion=self.canvas.bbox("all"))

    def mark_zones(self):
        self.canvas.bind("<ButtonPress-1>", self.start_marking_zone)
        self.template_zones = {}

    def start_marking_zone(self, event):
        self.start_x = self.canvas.canvasx(event.x)
        self.start_y = self.canvas.canvasy(event.y)
        self.rect = self.canvas.create_rectangle(self.start_x, self.start_y, self.start_x, self.start_y, outline='red')
        self.canvas.bind("<B1-Motion>", self.adjust_zone)
        self.canvas.bind("<ButtonRelease-1>", self.ask_zone_name)

    def adjust_zone(self, event):
        cur_x = self.canvas.canvasx(event.x)
        cur_y = self.canvas.canvasy(event.y)
        self.canvas.coords(self.rect, self.start_x, self.start_y, cur_x, cur_y)

    def ask_zone_name(self, event):
        self.canvas.unbind("<ButtonPress-1>")
        zone_name = simpledialog.askstring("Nombre de Zona", "Ingrese el nombre para esta zona:")
        if zone_name:
            zone_coords = self.canvas.coords(self.rect)
            self.template_zones[zone_name] = zone_coords
            self.create_textbox(zone_name)
        self.canvas.bind("<ButtonPress-1>", self.start_marking_zone)
        if not messagebox.askyesno("Pregunta", "¿Desea marcar otra zona?"):
            self.save_zones()

    def create_textbox(self, zone_name):
        label = tk.Label(self.buttons_frame, text=zone_name)
        label.grid(row=len(self.textboxes) + 6, column=0, pady=5)
        textbox = tk.Entry(self.buttons_frame)
        textbox.grid(row=len(self.textboxes) + 6, column=1, pady=5)
        self.textboxes[zone_name] = textbox

    def save_zones(self):
        self.canvas.unbind("<ButtonPress-1>")
        self.canvas.unbind("<B1-Motion>")
        self.canvas.unbind("<ButtonRelease-1>")
        self.btn_mark_zones.config(state=tk.DISABLED)
        self.btn_save_zones.config(state=tk.NORMAL)
        print("Zonas guardadas:")
        for zone_name, zone_coords in self.template_zones.items():
            if zone_name:  
                print(f"Nombre: {zone_name}, Coordenadas: {zone_coords}")
                conn = sqlite3.connect('gloria1.db')
                c = conn.cursor()
                plantilla_id = self.template_ids[self.template_var.get()]  # Obtener el plantilla_id de la plantilla actual
                c.execute('INSERT INTO ZonasMarcadas (plantilla_id, nombre_zona, coordenadas) VALUES (?, ?, ?)', (plantilla_id, zone_name, str(zone_coords)))
                conn.commit()
                conn.close()
        self.btn_save_zones.config(state=tk.DISABLED)

    def save_template(self):
        if not self.template_zones:
            messagebox.showwarning("Advertencia", "No hay zonas marcadas para guardar.")
            return
        template_name = simpledialog.askstring("Nombre de Plantilla", "Ingrese el nombre para la plantilla:")
        if not template_name:
            messagebox.showwarning("Advertencia", "Debe ingresar un nombre para la plantilla.")
            return
        self.templates[template_name] = self.template_zones
        messagebox.showinfo("Información", f"Plantilla '{template_name}' guardada correctamente.")
        self.update_template_menu()
        self.clear_canvas()

    def update_template_menu(self):
        menu = self.template_menu["menu"]
        menu.delete(0, "end")  
        for template_name in self.templates.keys():
            menu.add_command(label=template_name, command=lambda name=template_name: self.load_template_zones(name))

    def load_template_zones(self, template_name):
        self.canvas.delete("all") 
        self.show_template_image()
        self.template_zones = self.templates[template_name]
        for zone_name, zone_coords in self.template_zones.items():
            self.canvas.create_rectangle(zone_coords, outline='red')  
            extracted_text = self.extract_text_from_zone(zone_name, zone_coords)
            if zone_name in self.textboxes:
                self.textboxes[zone_name].insert(tk.END, extracted_text)

    def extract_text_from_zone(self, zone_name, zone_coords):
        try:
            doc = fitz.open(self.selected_file)
            text = ""
            for page in doc:
                zone_rect = fitz.Rect(zone_coords)
                pix = page.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                cropped_img = img.crop((zone_rect.x0, zone_rect.y0, zone_rect.x1, zone_rect.y1))
                
                # Convertir la imagen a escala de grises
                cropped_img = cropped_img.convert('L')
                
                # Aplicar OCR a la imagen en escala de grises
                text += pytesseract.image_to_string(cropped_img, lang='spa')
            return text
        except Exception as e:
            print(f"Error al extraer texto de la zona '{zone_name}': {e}")
            return ""

    def load_file(self):
        self.selected_file = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if self.selected_file:
            self.btn_select_template.config(state=tk.NORMAL)
            self.show_template_image()  
            
    def clear_canvas(self):
        self.canvas.delete("all")

def main():
    root = tk.Tk()
    app = TextExtractorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
